In [385]:
# Import Libraries 
import os
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [386]:
# read in data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [387]:
# drop columns that are not needed
train.drop(['Property_ID'], axis=1, inplace=True)
testPrpId = y = test['Property_ID']
test.drop(['Property_ID'], axis=1, inplace=True)

In [388]:
# split into X and y
y = train['Habitability_score']
train.drop(['Habitability_score'], axis=1, inplace=True)

In [389]:
# train.Neighborhood_Review = train.Neighborhood_Review.round()
# test.Neighborhood_Review = test.Neighborhood_Review.round()

In [390]:
# Converting string columns to numeric
categorical_features = ['Property_Type', 'Furnishing', 'Frequency_of_Powercuts', 'Power_Backup', 'Water_Supply', 'Crime_Rate', 'Dust_and_Noise']
labelencoder = LabelEncoder()
for feature in categorical_features:
    train[feature] = labelencoder.fit_transform(train[feature])
    train[feature].fillna(train[feature].mean(), inplace = True)
    test[feature] = labelencoder.fit_transform(test[feature])
    test[feature].fillna(test[feature].mean(), inplace = True)

# impute missing values
numerical_features = ['Property_Area', 'Number_of_Windows', 'Number_of_Doors','Traffic_Density_Score','Air_Quality_Index','Neighborhood_Review']
for feature in numerical_features:
    train[feature].fillna(train[feature].mean(), inplace = True)
    test[feature].fillna(test[feature].mean(), inplace = True)

In [391]:
train.head()

,Property_Type,Property_Area,Number_of_Windows,Number_of_Doors,Furnishing,Frequency_of_Powercuts,Power_Backup,Water_Supply,Traffic_Density_Score,Crime_Rate,Dust_and_Noise,Air_Quality_Index,Neighborhood_Review
0,1,106,3.923768,1,1,0,1,3,5.89,1,2,90.0,3.86
1,1,733,2.000000,2,2,1,1,2,4.37,3,2,96.0,3.55
2,1,737,4.000000,2,0,0,1,3,7.45,1,2,121.0,3.81
3,1,900,3.000000,2,2,2,2,3,6.16,2,2,100.0,1.34
4,2,2238,14.000000,6,0,0,1,0,5.46,3,2,116.0,4.77


In [392]:
# pipeline for categorical variables
categorical_features = ['Property_Type', 'Furnishing', 'Frequency_of_Powercuts', 'Power_Backup', 'Water_Supply', 'Crime_Rate', 'Dust_and_Noise']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [393]:
# pipeline for numerical variables
numerical_features = ['Property_Area', 'Number_of_Windows', 'Number_of_Doors','Traffic_Density_Score','Air_Quality_Index','Neighborhood_Review']
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

In [394]:
# transform full pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])
# Use the full pipeline to transform both training and test sets
preprocessor.fit(train)
train = preprocessor.transform(train)
test = preprocessor.transform(test)


In [395]:
# feature selection


In [396]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)


In [397]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

In [398]:
# Define models
models = [
    RandomForestRegressor(),
]

In [399]:
# Evaluate each model in turn
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Model: %s" % model.__class__.__name__)
    print("R-squared: %f" % r2_score(y_test, y_pred))
    print("\n")

Model: RandomForestRegressor
R-squared: 0.822313




In [400]:
# Final model
final_model = RandomForestRegressor()

In [402]:
# make final predictions
final_model.fit(train, y)
final_predictions = final_model.predict(test)

In [ ]:
# save results to file
results = pd.DataFrame({'Property_ID': testPrpId, 'Habitability_score': final_predictions})

filename = "submission.csv"

if os.path.exists(filename):
  os.remove(filename)
results.to_csv(filename, index=False,header=True, mode='w')